In [23]:
# first run installations and data download
! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
! mkdir -p './storage/commits'


zsh:1: command not found: pip


In [24]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [25]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/commits"
    )
    commit_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [26]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [27]:
if not index_loaded:
    # load data
    commit_docs = SimpleDirectoryReader(
        input_files=["./0001-adding-a-better-calculator.patch"]
    ).load_data()

    # build index
    vector_store_commit = MilvusVectorStore(dim=1536, collection_name="commit", overwrite=True)
    
    storage_context_commit = StorageContext.from_defaults(vector_store=vector_store_commit)
    commit_index = VectorStoreIndex.from_documents(commit_docs, storage_context=storage_context_commit)


    # persist index
    commit_index.storage_context.persist(persist_dir="./storage/commit")


In [28]:
commit_engine = commit_index.as_query_engine(similarity_top_k=3)

In [29]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=commit_engine,
        metadata=ToolMetadata(
            name="commit_summary",
            description=(
                "Provides summary information about a commit patch.  "
                # "Need to manually run 'git format-patch -1 to generate the patch file."
            ),
        ),
    ),
]

In [30]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [31]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [47]:
agent1_response = agent.chat("Describe in great detail what changes were made in this patch?")
print(str(agent1_response))

Thought: I need to use a tool to help me answer the question.
Action: commit_summary
Action Input: {'input': 'describe changes in patch'}
Observation: The changes in the patch include the addition of a new Jupyter notebook file named "better_calculator_agent.ipynb" with 212 insertions. This notebook contains code cells defining functions for multiplication, addition, and squaring integers. Additionally, it imports necessary modules and initializes an OpenAI model. The patch also includes interactions where calculations are performed step by step using the defined functions, demonstrating the order of operations and the results of the calculations.
Thought: I can answer without using any more tools.
Answer: The changes in the patch include the addition of a new Jupyter notebook file named "better_calculator_agent.ipynb" with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model "gpt-4" and demonstrates step-by-step calcula

In [38]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [52]:
def set_title(commit_body) -> str:
    """Takes in a commit body and returns a recommended blog title"""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.completion_to_prompt("generate a blog title of 10 words or less for the following code changes : " + commit_body)
    return response


title_tool = FunctionTool.from_defaults(fn=set_title)

In [53]:
llm = OpenAI(model="gpt-3.5-turbo")
#  llm = OpenAI(model="gpt-4")
title_agent = ReActAgent.from_tools([title_tool], llm=llm, verbose=True)

In [54]:
response_gen = title_agent.chat(str(agent1_response))
print(str(response_gen))

Thought: I need to use a tool to help me generate a blog title for this commit.
Action: set_title
Action Input: {'commit_body': "The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations."}
Observation: generate a blog title of 10 words or less for the following code changes : The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations.
Thought: I can answer without using any more tools.
Answer: "Introducing Better Calculator Agent: Interactive Math Operations w

In [55]:
def set_summary(commit_body) -> str:
    """Takes in a commit body and returns a recommended commit summary"""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.completion_to_prompt("generate a blog summary of 200 words for the following code changes : " + commit_body)
    return response


summary_tool = FunctionTool.from_defaults(fn=set_summary)

In [56]:
summary_agent = ReActAgent.from_tools([summary_tool], llm=llm, verbose=True)

In [57]:
# response_summary = summary_agent.stream_chat(str(agent1_response))
# response_gen.print_response_stream()

response_summary = summary_agent.chat(str(agent1_response))
print(str(response_summary))

Thought: I need to use a tool to generate a commit summary for the changes described in the patch.
Action: set_summary
Action Input: {'commit_body': "The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations."}
Observation: generate a blog summary of 200 words for the following code changes : The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations.
Thought: I can generate a blog summary without using any more tools.
Answer: The patch introduces a new Jupyter

In [65]:
note_file = "notes.txt"  # Directly assign the file name to the variable

def save_note(note):
    # Check if the file does not exist before opening it in write mode
    if not os.path.exists(note_file):
        open(note_file, "w").close()  # Open and immediately close the file to create it if it doesn't exist

    # Open the file in append mode and write the note
    with open(note_file, "a") as f:
        f.writelines([note + "\n"])

    return "note saved"

In [66]:
# blog_agent = ReActAgent.from_tools([summary_tool, title_tool], llm=llm, verbose=True)

def write_blog():
    title = title_agent.chat(str(agent1_response))
    summary = summary_agent.chat(str(agent1_response))
    return title, summary

title, summary = write_blog()
save_note(str(title))
save_note(str(summary))

Thought: I need to use a tool to help me answer the question.
Action: set_title
Action Input: {'commit_body': "The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations."}
Observation: generate a blog title of 10 words or less for the following code changes : The changes in the patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with functions for multiplication, addition, and squaring integers. It also includes code to interact with the OpenAI model 'gpt-4' and demonstrates step-by-step calculations for various mathematical operations.
Thought: I can answer without using any more tools.
Answer: "Enhanced Calculator Agent: Math Operations with GPT-4 in Jupyter"
Thought: I need to

TypeError: unsupported operand type(s) for +: 'AgentChatResponse' and 'str'